<a href="https://colab.research.google.com/github/Sagu12/all-projects/blob/master/News_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
df= pd.read_json("/content/drive/MyDrive/enhanceduserdatabeta1.json", lines=True)

In [3]:
df.head(2)

,_index,_type,_id,_score,_source
0,enhanceduserdatabeta1,core2,AXGicw8zutugd-ZqVCZf,1,"{'publisher_id': '49491', 'date': '2020-04-22'..."
1,enhanceduserdatabeta1,core2,AXGidE5mCFpND4aqB9Bq,1,"{'publisher_id': '49491', 'date': '2020-04-22'..."


In [4]:
source=pd.json_normalize(df._source)

In [5]:
con= pd.concat([df, source], axis=1)

In [6]:
con= con.drop("_source", axis=1)

In [7]:
con["refcurrentoriginal"]= con["refcurrentoriginal"].astype("str")

In [8]:
con['result'] = con['refcurrentoriginal'].map(lambda x: x.lstrip('https://m. www').rstrip('aAbBcC'))

In [ ]:
ps= WordNetLemmatizer()
corpus=[]
for i in range(0,len(con)):
  print(i)
  review= re.sub("[^a-zA-Z]", " ", con["result"][i])
  review= review.lower()
  review= review.split()
  review= [ps.lemmatize(word) for word in review if not word in stopwords.words("english")]
  review= " ".join(review)
  corpus.append(review)

In [10]:
con["actual_news"]= corpus

con.drop(["result", "refcurrentoriginal"], axis=1, inplace=True)

In [11]:
con_updated= con[["ip","engagementTime","gender","visitor_id","actual_news","audience_segment"]]

In [12]:
con_updated.head(2)

,ip,engagementTime,gender,visitor_id,actual_news,audience_segment
0,"2409:4052:887:1836:1d8f:8a33:34b5:f29d, 64.233...",6.0,female,e9b7951d-f0fd-4c1b-88b3-4289eda2fb78,aajtak khabare ab tak story caste based census...,
1,"2405:204:1299:b635:7e57:1081:c615:95d9, 64.233...",1.0,male,754cc585-b30e-4b62-9288-9fedfdc42684,aajtak karyakrams da tak video crime woman stop,


In [13]:
con_updated.isnull().sum()

ip                     0
engagementTime      8116
gender              8129
visitor_id             0
actual_news            0
audience_segment       0
dtype: int64

In [14]:
con_updated= con_updated[con_updated.actual_news != "nan"]

In [15]:
con_updated.shape

(99471, 6)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectornews = vectorizer.fit_transform(con_updated["actual_news"])

In [17]:
from sklearn.decomposition import LatentDirichletAllocation

In [18]:
LDA = LatentDirichletAllocation(n_components=23, random_state=42)

In [19]:
topic_values = LDA.fit_transform(vectornews)

In [20]:
con_updated['audience_segment_updated'] = topic_values.argmax(axis=1)

In [21]:
con_updated.head(2)

,ip,engagementTime,gender,visitor_id,actual_news,audience_segment,audience_segment_updated
0,"2409:4052:887:1836:1d8f:8a33:34b5:f29d, 64.233...",6.0,female,e9b7951d-f0fd-4c1b-88b3-4289eda2fb78,aajtak khabare ab tak story caste based census...,,21
1,"2405:204:1299:b635:7e57:1081:c615:95d9, 64.233...",1.0,male,754cc585-b30e-4b62-9288-9fedfdc42684,aajtak karyakrams da tak video crime woman stop,,21


In [23]:
con_updated.shape

(99471, 7)

In [22]:
con_updated['actual_news'].apply(lambda x: len(x.split(' '))).sum()


1000361

# <i> Using Different Machine Learning models to see the best fit model out of them. Basically, here we will be classifying the news based on the topic modelling we created.  So, whenever a new news article will be fed to the machine learning model then it will be classified based on the new segments that we have created.

In [24]:
new_df= con_updated[["actual_news","audience_segment_updated"]]

In [52]:
new_df.head(30)

,actual_news,audience_segment_updated
0,aajtak khabare ab tak story caste based census...,21
1,aajtak karyakrams da tak video crime woman stop,21
2,aajtak national video sapna chaudhary dance sh...,13
3,aajtak movie masala news film story shahid kap...,6
4,aajtak karyakrams kismat konnection video kism...,2
5,aajtak movie masala news film story pulkit sam...,6
6,aajtak movie masala news film story shah rukh ...,6
7,aajtak aajtak special ayodhya case verdict sto...,21
8,aajtak movie masala viral video story pawan si...,6
9,aajtak movie masala news film story vidyut jam...,13


In [57]:
new_df.iloc[25]

actual_news                 aajtak style lifestyle story story india today...
audience_segment_updated                                                   16
Name: 25, dtype: object

# <i> Splitting the data into train and test for our machine learning model

In [38]:
X= new_df.actual_news
y= new_df.audience_segment_updated

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((74603,), (24868,), (74603,), (24868,))

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import cross_validate, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

In [42]:
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [43]:
# Creating Models
models = [('Logistic Regression', LogisticRegression(max_iter=500)),('Random Forest', RandomForestClassifier()),
          ('Linear SVC', LinearSVC()), ('Multinomial NaiveBayes', MultinomialNB()), ('SGD Classifier', SGDClassifier())]

In [44]:
names = []
results = []
model = []
for name, clf in models:
    pipe = Pipeline([('vect', CountVectorizer(max_features=30000, ngram_range=(1, 2))),
                    ('tfidf', TfidfTransformer()),
                    (name, clf),
                    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    
    names.append(name)
    results.append(accuracy)
    model.append(pipe)
    
    msg = "%s: %f" % (name, accuracy)
    print(msg)

Logistic Regression: 0.969358
Random Forest: 0.969278
Linear SVC: 0.975873
Multinomial NaiveBayes: 0.960190
SGD Classifier: 0.965458


# <i> Above we can see that Logistic Regression and Linear SVC have performed well out of the other models so we will be using these two models for our testing purpose

In [46]:
import pickle

In [47]:
# Logistic Regression
filename = 'model_lr.sav'
pickle.dump(model[0], open(filename, 'wb'))

# Linear SVC
filename = 'model_lin_svc.sav'
pickle.dump(model[2], open(filename, 'wb'))

In [48]:
lr_model = pickle.load(open('model_lin_svc.sav', 'rb'))

In [60]:
text1 = 'Could have been best all-rounder India ever produced in ODIs: Irfan Pathan'

text2 = "Ashwin Kkumar dances to Kamal Haasan's Annathe song on a treadmill. Actor is proud"

In [61]:
print(lr_model.predict([text1, text2]))

[16  6]


# <i> Here above we can see that our model of linear SVC is doing pretty much good in terms of prediction and therefore this model can be used for news classification purposes.

# <i> Checking our model on some news posted in India Today's website

In [62]:
t1= "9-year-old girl plays synthesizer while undergoing brain surgery"

t2= "Anand Mahindra is bowled over by Lucky Ali's O Sanam performance"

t3= "J&K DDC polls: Phase-V ends with over 51% polling, says SEC Sharma"

In [63]:
print(lr_model.predict([t1, t2, t3]))

[21  4 13]
